"""
# Projet Python : Simulation d'une usine de fabrication de jouets

Ce notebook contient une correction détaillée du projet de simulation d'une usine, en respectant les consignes suivantes :
- Utilisation de la programmation orientée objet (POO).
- Gestion des files d'attente (queue) et des piles (stack).
- Utilisation de la bibliothèque `itertools`, du module `logging`, de la récursivité, et des fonctions `map`, `filter`, `reduce`.
- Utilisation d'expressions régulières (regex) pour valider et analyser les identifiants des produits.
- Simulation de pannes pour ajouter du réalisme.

Chaque classe et fonction est expliquée en détail avant son implémentation, avec un focus sur les concepts Python utilisés.

---

## Imports nécessaires

Nous commençons par importer les bibliothèques nécessaires :
- `collections` : Pour les structures de données comme `deque` (utilisé pour les files d'attente).
- `itertools` : Pour générer des séquences infinies ou limitées de produits.
- `logging` : Pour enregistrer les événements dans un fichier de logs.
- `random` : Pour simuler des événements aléatoires (temps de traitement, défauts, pannes).
- `time` : Pour simuler des délais.
- `re` : Pour les expressions régulières.
- `functools.reduce` : Pour utiliser la fonction `reduce`.



In [28]:
import collections
import logging
import random
import time
import re
from functools import reduce

## Configuration du logging

Le module `logging` est utilisé pour enregistrer tous les événements importants de la simulation (par exemple, le traitement d'un produit, une panne, un défaut, etc.). Les logs sont écrits dans un fichier `usine.log` avec un format clair incluant la date, le niveau de log (INFO, WARNING, ERROR), et le message.

### Concepts utilisés :
- `logging.basicConfig` : Configure le système de logging global.
- Niveaux de logging : `INFO` pour les informations générales, `WARNING` pour les avertissements (par exemple, une panne), `ERROR` pour les erreurs graves (par exemple, un produit rejeté).

In [20]:
# Configuration du logging
logging.basicConfig(
    filename='usine.log',
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)

## Classe `Produit`

La classe `Produit` représente un jouet fabriqué dans l'usine. Chaque produit a un identifiant unique, un statut, un temps total de production, et un suivi des étapes effectuées. Elle inclut également des méthodes pour valider et analyser l'identifiant avec des expressions régulières (regex).

### Attributs :
- `id` : Identifiant unique sous la forme `"TOY-XXX-COLOR"` (par exemple, `"TOY-001-RED"`).
- `etapes_effectuees` : Liste des étapes terminées (par exemple, `["assemblage", "peinture"]`).
- `status` : Statut du produit (`"en cours"`, `"fini"`, ou `"rejeté"`).
- `temps_total` : Temps total de production (en secondes simulées).
- `tentatives` : Nombre de tentatives de correction en cas de défaut.

### Méthodes :
- `__init__` : Initialise un produit avec un numéro de série et une couleur.
- `__str__` : Fournit une représentation sous forme de chaîne pour faciliter l'affichage.
- `valider_id` : Utilise une regex pour valider le format de l'identifiant.
- `extraire_numero_serie` : Utilise une regex pour extraire le numéro de série de l'identifiant.

### Concepts utilisés :
- **POO** : Encapsulation des données et méthodes dans une classe.
- **Regex** : Utilisation de `re.match` pour valider le format et `re.search` pour extraire des informations.


In [21]:
class Produit:
    def __init__(self, numero_serie, couleur):
        self.id = f"TOY-{numero_serie:03d}-{couleur.upper()}"
        self.etapes_effectuees = []  # Liste des étapes terminées
        self.status = "en cours"
        self.temps_total = 0  # Temps total de production
        self.tentatives = 0  # Nombre de tentatives pour corriger les défauts

    def __str__(self):
        return f"{self.id} (Status: {self.status}, Temps: {self.temps_total}s, Tentatives: {self.tentatives})"

    def valider_id(self):
        """Valide le format de l'identifiant avec une regex."""
        pattern = r"TOY-\d{3}-[A-Z]+"
        return bool(re.match(pattern, self.id))

    def extraire_numero_serie(self):
        """Extrait le numéro de série de l'identifiant avec une regex."""
        match = re.search(r"TOY-(\d{3})-[A-Z]+", self.id)
        return int(match.group(1)) if match else None

## Classe `Station`

La classe `Station` représente une étape de production dans l'usine (par exemple, assemblage, peinture, contrôle qualité, emballage). Chaque station a une file d'attente pour gérer les produits en attente et peut tomber en panne de manière aléatoire, ce qui bloque le traitement temporairement.

### Attributs :
- `nom` : Nom de la station (par exemple, `"assemblage"`).
- `temps_traitement_base` : Temps de traitement moyen (en secondes simulées).
- `file_attente` : File d'attente (queue) pour stocker les produits en attente, implémentée avec `collections.deque`.
- `en_panne` : Booléen indiquant si la station est en panne.
- `temps_restant_panne` : Temps restant avant la fin de la panne.
- `proba_panne` : Probabilité qu'une panne se produise à chaque traitement (par défaut, 5 %).
- `duree_panne_min` et `duree_panne_max` : Durées minimale et maximale d'une panne (par défaut, 5 à 10 secondes simulées).

### Méthodes :
- `__init__` : Initialise une station avec ses paramètres.
- `ajouter_produit` : Ajoute un produit à la file d'attente.
- `verifier_panne` : Vérifie si une panne se produit ou met à jour l'état de la panne en cours.
- `traiter_produit` : Traite le produit en tête de la file d'attente, en tenant compte des pannes.

### Concepts utilisés :
- **POO** : Encapsulation et gestion des états (par exemple, `en_panne`).
- **Queue** : Utilisation de `collections.deque` pour une file FIFO (First In, First Out).
- **Simulation de pannes** : Utilisation de `random` pour simuler des événements aléatoires et `time.sleep` pour simuler des délais.
- **Logging** : Enregistrement des événements (ajout de produit, traitement, pannes) dans les logs.


In [ ]:

class Station:
    def __init__(self, nom, temps_traitement_base, proba_panne=0.05, duree_panne_min=5, duree_panne_max=10):
        self.nom = nom
        self.temps_traitement_base = temps_traitement_base
        self.file_attente = collections.deque()  # File d'attente (queue)
        # Attributs pour les pannes
        self.en_panne = False
        self.temps_restant_panne = 0
        self.proba_panne = proba_panne  # Probabilité de panne à chaque traitement
        self.duree_panne_min = duree_panne_min
        self.duree_panne_max = duree_panne_max

    def ajouter_produit(self, produit):
        """Ajoute un produit à la file d'attente."""
        self.file_attente.append(produit)
        logging.info(f"Produit {produit.id} ajouté à la file d'attente de {self.nom}.")

    def verifier_panne(self):
        """Vérifie si une panne se produit ou met à jour l'état de la panne."""
        if self.en_panne:
            self.temps_restant_panne -= 1  # Décrémente le temps restant
            if self.temps_restant_panne <= 0:
                self.en_panne = False
                logging.info(f"La station {self.nom} est réparée et reprend son fonctionnement.")
        elif random.random() < self.proba_panne:  # Probabilité de panne
            self.en_panne = True
            self.temps_restant_panne = random.uniform(self.duree_panne_min, self.duree_panne_max)
            logging.warning(f"La station {self.nom} est en panne pour {self.temps_restant_panne:.2f}s.")

    def traiter_produit(self):
        """Traite le produit en tête de la file d'attente, en tenant compte des pannes."""
        # Vérifie si une panne se produit ou est en cours
        self.verifier_panne()
        if self.en_panne:
            logging.warning(f"La station {self.nom} est en panne, aucun produit traité.")
            return None

        if not self.file_attente:
            # Pas de produit à traiter
            return None
        produit = self.file_attente.popleft()
        temps_traitement = random.uniform(self.temps_traitement_base * 0.5, self.temps_traitement_base * 1.5)
        time.sleep(temps_traitement / 10)  # Simule un délai (accéléré pour le test)
        produit.temps_total += temps_traitement
        produit.etapes_effectuees.append(self.nom)
        logging.info(f"Produit {produit.id} a terminé l'étape {self.nom} en {temps_traitement:.2f}s.")
        return produit

## Classe `PileDefauts`

La classe `PileDefauts` gère une pile (stack) de produits défectueux détectés au contrôle qualité. Les produits défectueux sont empilés et renvoyés à une étape précédente pour correction, avec une limite au nombre de tentatives.

### Attributs :
- `pile` : Pile (stack) pour stocker les produits défectueux, implémentée comme une liste.

### Méthodes :
- `__init__` : Initialise une pile vide.
- `ajouter_defaut` : Ajoute un produit défectueux à la pile.
- `traiter_defaut` : Dépile un produit et le renvoie à une étape précédente, en utilisant une récursivité simple pour gérer les tentatives.

### Concepts utilisés :
- **Stack** : Utilisation d'une liste comme pile LIFO (Last In, First Out) avec `append` pour empiler et `pop` pour dépiler.
- **Récursivité** : La méthode `traiter_defaut` est conçue pour illustrer une récursivité simple, bien que dans ce cas elle ne soit pas profondément récursive (elle ne s'appelle pas elle-même directement, mais illustre le concept de renvoi à une étape précédente).
- **Logging** : Enregistrement des événements (ajout à la pile, renvoi pour correction, rejet).


In [27]:
class PileDefauts:
    def __init__(self):
        self.pile = []  # Pile (stack) pour les produits défectueux

    def ajouter_defaut(self, produit):
        """Ajoute un produit défectueux à la pile."""
        self.pile.append(produit)
        logging.warning(f"Produit {produit.id} marqué comme défectueux, ajouté à la pile.")

    def traiter_defaut(self, usine, etape_retour):
        """Traite récursivement un produit défectueux."""
        if not self.pile:
            # Si la pile est vide, il n'y a plus de produits défectueux à traiter
            return
        produit = self.pile.pop()  # Dépile le produit (LIFO)
        produit.tentatives += 1
        if produit.tentatives >= 3:
            produit.status = "rejeté"
            logging.error(f"Produit {produit.id} rejeté après 3 tentatives.")
            return
        logging.info(f"Produit {produit.id} renvoyé à l'étape {etape_retour} pour correction (tentative {produit.tentatives + 1}).")
        usine.stations[etape_retour].ajouter_produit(produit)

## Classe `Usine`

La classe `Usine` est la classe principale qui orchestre toute la simulation. Elle gère les stations de production, la pile des défauts, et les produits finis, et produit un rapport final à la fin de la simulation.

### Attributs :
- `stations` : Dictionnaire des stations de production, avec les noms comme clés et les instances de `Station` comme valeurs.
- `pile_defauts` : Instance de `PileDefauts` pour gérer les produits défectueux.
- `produits_finis` : Liste des produits terminés ou rejetés.

### Méthodes :
- `__init__` : Initialise les stations, la pile des défauts, et la liste des produits finis.
- `generer_produits` : Génère un flux de produits à traiter.
- `simuler_production` : Simule le flux de production à travers les stations, en tenant compte des pannes et des défauts.
- `generer_rapport` : Analyse les données de production avec `map`, `filter`, et `reduce` pour produire un rapport final.

### Concepts utilisés :
- **POO** : Orchestration de plusieurs objets (stations, pile des défauts, produits).
- **Itertools** : Utilisation de `itertools.count` et `itertools.islice` pour générer un flux limité de produits.
- **Map, Filter, Reduce** : Utilisation dans `generer_rapport` pour analyser les données de production.
- **Logging** : Enregistrement des événements dans les logs.


In [37]:
class Usine:
    def __init__(self):
        # Les stations sont initialisées avec une probabilité de panne
        self.stations = {
            "assemblage": Station("assemblage", 2, proba_panne=0.05),
            "peinture": Station("peinture", 3, proba_panne=0.05),
            "controle_qualite": Station("controle_qualite", 1, proba_panne=0.02),  # Moins de pannes ici
            "emballage": Station("emballage", 1, proba_panne=0.05)
        }
        # Pile de produits défectueux
        self.pile_defauts = PileDefauts()
        # Liste des produits finis
        self.produits_finis = []

    def generer_produits(self, nombre_produits):
        """Génère un flux continu de produits."""
        couleurs = ["RED", "BLUE", "GREEN"]
        for i in range(1, nombre_produits + 1):
            couleur = random.choice(couleurs)
            produit = Produit(i, couleur)
            if produit.valider_id():
                # Ajoute le produit à la station d'assemblage
                self.stations["assemblage"].ajouter_produit(produit)
            else:
                logging.error(f"Produit {produit.id} invalide, ignoré.")

    def simuler_production(self):
        """Simule le flux de production à travers les stations."""
        etapes = ["assemblage", "peinture", "controle_qualite", "emballage"]
        # any(station.file_attente for station in self.stations.values()) : tant qu'il reste des produits en attente
        # self.pile_defauts.pile : tant qu'il reste des produits défectueux à traiter
        while any(station.file_attente for station in self.stations.values()) or self.pile_defauts.pile:
            # On itère sur les étapes de production tant qu'il reste des produits à traiter
            for i, etape in enumerate(etapes):
                # Traite le produit en cours à chaque étape
                produit = self.stations[etape].traiter_produit()
                # produit est None si la file d'attente est vide ou si la station est en panne
                if produit:
                    if etape == "controle_qualite":
                        if random.random() < 0.2:  # 20% de chance de défaut
                            self.pile_defauts.ajouter_defaut(produit)
                            # Traite le défaut à l'étape précédente (peinture ou assemblage aléatoirement)
                            self.pile_defauts.traiter_defaut(self, random.choice(["assemblage", "peinture"]))
                        else:
                            # Passe à l'étape suivante si le produit est conforme
                            self.stations[etapes[i + 1]].ajouter_produit(produit)
                    elif etape == "emballage":
                        produit.status = "fini"
                        self.produits_finis.append(produit)
                    elif i < len(etapes) - 1:
                        self.stations[etapes[i + 1]].ajouter_produit(produit)

    def generer_rapport(self):
        """Génère un rapport final avec map, filter, reduce."""
        print("\n=== Rapport final ===")
        # Nombre total de jouets produits
        total_produits = len(self.produits_finis)
        print(f"Nombre total de jouets produits : {total_produits}")

        # Pourcentage de jouets défectueux
        defauts = list(filter(lambda p: p.tentatives > 0, self.produits_finis))
        pourcentage_defauts = (len(defauts) / total_produits * 100) if total_produits > 0 else 0
        print(f"Pourcentage de jouets défectueux : {pourcentage_defauts:.2f}%")

        # Temps moyen de production
        temps_moyen = reduce(lambda x, y: x + y.temps_total, self.produits_finis, 0) / total_produits if total_produits > 0 else 0
        print(f"Temps moyen de production par jouet : {temps_moyen:.2f}s")

        # Nombre de jouets rejetés
        rejetes = list(filter(lambda p: p.status == "rejeté", self.produits_finis))
        print(f"Nombre de jouets rejetés : {len(rejetes)}")

        # Exemple d'utilisation de map : liste des numéros de série
        numeros_series = list(map(lambda p: p.extraire_numero_serie(), self.produits_finis))
        print(f"Numéros de série des produits finis : {numeros_series}")

## Fonction `main`

La fonction `main` est la fonction principale qui lance la simulation. Elle crée une instance de `Usine`, génère un certain nombre de produits, exécute la simulation, et affiche le rapport final.

### Concepts utilisés :
- **POO** : Création et utilisation d'une instance de `Usine`.
- **Simulation** : Orchestration de l'ensemble du processus de production.

In [35]:
def main():
    usine = Usine()
    nombre_produits = 10  # Simuler 10 produits
    print(f"Simulation de la production de {nombre_produits} jouets...")
    usine.generer_produits(nombre_produits)
    usine.simuler_production()
    usine.generer_rapport()

In [38]:
# Point d'entrée du script
main()

Simulation de la production de 10 jouets...

=== Rapport final ===
Nombre total de jouets produits : 10
Pourcentage de jouets défectueux : 40.00%
Temps moyen de production par jouet : 9.23s
Nombre de jouets rejetés : 0
Numéros de série des produits finis : [2, 3, 4, 6, 7, 8, 1, 5, 9, 10]


# Aperçu du fichier de log

```log
2025-03-11 17:54:06,334 - INFO - Produit TOY-001-BLUE ajouté à la file d'attente de assemblage.
2025-03-11 17:54:06,335 - INFO - Produit TOY-002-RED ajouté à la file d'attente de assemblage.
2025-03-11 17:54:06,335 - INFO - Produit TOY-003-BLUE ajouté à la file d'attente de assemblage.
2025-03-11 17:54:06,335 - INFO - Produit TOY-004-RED ajouté à la file d'attente de assemblage.
2025-03-11 17:54:06,335 - INFO - Produit TOY-005-RED ajouté à la file d'attente de assemblage.
2025-03-11 17:54:06,335 - INFO - Produit TOY-006-BLUE ajouté à la file d'attente de assemblage.
2025-03-11 17:54:06,335 - INFO - Produit TOY-007-GREEN ajouté à la file d'attente de assemblage.
2025-03-11 17:54:06,335 - INFO - Produit TOY-008-GREEN ajouté à la file d'attente de assemblage.
2025-03-11 17:54:06,335 - INFO - Produit TOY-009-RED ajouté à la file d'attente de assemblage.
2025-03-11 17:54:06,335 - INFO - Produit TOY-010-BLUE ajouté à la file d'attente de assemblage.
2025-03-11 17:54:06,532 - INFO - Produit TOY-001-BLUE a terminé l'étape assemblage en 1.92s.
2025-03-11 17:54:06,532 - INFO - Produit TOY-001-BLUE ajouté à la file d'attente de peinture.
2025-03-11 17:54:06,763 - INFO - Produit TOY-001-BLUE a terminé l'étape peinture en 2.28s.
2025-03-11 17:54:06,763 - INFO - Produit TOY-001-BLUE ajouté à la file d'attente de controle_qualite.
2025-03-11 17:54:06,878 - INFO - Produit TOY-001-BLUE a terminé l'étape controle_qualite en 1.10s.
2025-03-11 17:54:06,879 - INFO - Produit TOY-001-BLUE ajouté à la file d'attente de emballage.
2025-03-11 17:54:07,029 - INFO - Produit TOY-001-BLUE a terminé l'étape emballage en 1.46s.
2025-03-11 17:54:07,147 - INFO - Produit TOY-002-RED a terminé l'étape assemblage en 1.14s.
2025-03-11 17:54:07,147 - INFO - Produit TOY-002-RED ajouté à la file d'attente de peinture.
2025-03-11 17:54:07,374 - INFO - Produit TOY-002-RED a terminé l'étape peinture en 2.23s.
2025-03-11 17:54:07,375 - INFO - Produit TOY-002-RED ajouté à la file d'attente de controle_qualite.
2025-03-11 17:54:07,443 - INFO - Produit TOY-002-RED a terminé l'étape controle_qualite en 0.63s.
2025-03-11 17:54:07,444 - INFO - Produit TOY-002-RED ajouté à la file d'attente de emballage.
2025-03-11 17:54:07,545 - INFO - Produit TOY-002-RED a terminé l'étape emballage en 0.99s.
2025-03-11 17:54:07,720 - INFO - Produit TOY-003-BLUE a terminé l'étape assemblage en 1.69s.
2025-03-11 17:54:07,720 - INFO - Produit TOY-003-BLUE ajouté à la file d'attente de peinture.
2025-03-11 17:54:08,024 - INFO - Produit TOY-003-BLUE a terminé l'étape peinture en 3.01s.
2025-03-11 17:54:08,024 - INFO - Produit TOY-003-BLUE ajouté à la file d'attente de controle_qualite.
2025-03-11 17:54:08,154 - INFO - Produit TOY-003-BLUE a terminé l'étape controle_qualite en 1.25s.
2025-03-11 17:54:08,154 - WARNING - Produit TOY-003-BLUE marqué comme défectueux, ajouté à la pile.
2025-03-11 17:54:08,154 - INFO - Produit TOY-003-BLUE renvoyé à l'étape assemblage pour correction (tentative 2).
2025-03-11 17:54:08,154 - INFO - Produit TOY-003-BLUE ajouté à la file d'attente de assemblage.
2025-03-11 17:54:08,455 - INFO - Produit TOY-004-RED a terminé l'étape assemblage en 2.96s.
2025-03-11 17:54:08,455 - INFO - Produit TOY-004-RED ajouté à la file d'attente de peinture.
2025-03-11 17:54:08,645 - INFO - Produit TOY-004-RED a terminé l'étape peinture en 1.87s.
2025-03-11 17:54:08,645 - INFO - Produit TOY-004-RED ajouté à la file d'attente de controle_qualite.
2025-03-11 17:54:08,795 - INFO - Produit TOY-004-RED a terminé l'étape controle_qualite en 1.46s.
2025-03-11 17:54:08,795 - INFO - Produit TOY-004-RED ajouté à la file d'attente de emballage.
2025-03-11 17:54:08,856 - INFO - Produit TOY-004-RED a terminé l'étape emballage en 0.55s.
2025-03-11 17:54:09,155 - INFO - Produit TOY-005-RED a terminé l'étape assemblage en 2.94s.
2025-03-11 17:54:09,155 - INFO - Produit TOY-005-RED ajouté à la file d'attente de peinture.
2025-03-11 17:54:09,541 - INFO - Produit TOY-005-RED a terminé l'étape peinture en 3.81s.
2025-03-11 17:54:09,541 - INFO - Produit TOY-005-RED ajouté à la file d'attente de controle_qualite.
2025-03-11 17:54:09,608 - INFO - Produit TOY-005-RED a terminé l'étape controle_qualite en 0.62s.
2025-03-11 17:54:09,608 - INFO - Produit TOY-005-RED ajouté à la file d'attente de emballage.
2025-03-11 17:54:09,738 - INFO - Produit TOY-005-RED a terminé l'étape emballage en 1.24s.
2025-03-11 17:54:09,892 - INFO - Produit TOY-006-BLUE a terminé l'étape assemblage en 1.50s.
2025-03-11 17:54:09,892 - INFO - Produit TOY-006-BLUE ajouté à la file d'attente de peinture.
2025-03-11 17:54:10,279 - INFO - Produit TOY-006-BLUE a terminé l'étape peinture en 3.82s.
2025-03-11 17:54:10,280 - INFO - Produit TOY-006-BLUE ajouté à la file d'attente de controle_qualite.
2025-03-11 17:54:10,410 - INFO - Produit TOY-006-BLUE a terminé l'étape controle_qualite en 1.26s.
2025-03-11 17:54:10,410 - INFO - Produit TOY-006-BLUE ajouté à la file d'attente de emballage.
2025-03-11 17:54:10,467 - INFO - Produit TOY-006-BLUE a terminé l'étape emballage en 0.55s.
2025-03-11 17:54:10,729 - INFO - Produit TOY-007-GREEN a terminé l'étape assemblage en 2.59s.
2025-03-11 17:54:10,729 - INFO - Produit TOY-007-GREEN ajouté à la file d'attente de peinture.
2025-03-11 17:54:11,112 - INFO - Produit TOY-007-GREEN a terminé l'étape peinture en 3.78s.
2025-03-11 17:54:11,114 - INFO - Produit TOY-007-GREEN ajouté à la file d'attente de controle_qualite.
2025-03-11 17:54:11,178 - INFO - Produit TOY-007-GREEN a terminé l'étape controle_qualite en 0.60s.
2025-03-11 17:54:11,179 - INFO - Produit TOY-007-GREEN ajouté à la file d'attente de emballage.
2025-03-11 17:54:11,275 - INFO - Produit TOY-007-GREEN a terminé l'étape emballage en 0.91s.
2025-03-11 17:54:11,414 - INFO - Produit TOY-008-GREEN a terminé l'étape assemblage en 1.33s.
2025-03-11 17:54:11,414 - INFO - Produit TOY-008-GREEN ajouté à la file d'attente de peinture.
2025-03-11 17:54:11,827 - INFO - Produit TOY-008-GREEN a terminé l'étape peinture en 4.08s.
2025-03-11 17:54:11,827 - INFO - Produit TOY-008-GREEN ajouté à la file d'attente de controle_qualite.
2025-03-11 17:54:11,916 - INFO - Produit TOY-008-GREEN a terminé l'étape controle_qualite en 0.84s.
2025-03-11 17:54:11,916 - INFO - Produit TOY-008-GREEN ajouté à la file d'attente de emballage.
2025-03-11 17:54:12,038 - INFO - Produit TOY-008-GREEN a terminé l'étape emballage en 1.20s.
2025-03-11 17:54:12,339 - INFO - Produit TOY-009-RED a terminé l'étape assemblage en 3.00s.
2025-03-11 17:54:12,339 - INFO - Produit TOY-009-RED ajouté à la file d'attente de peinture.
2025-03-11 17:54:12,512 - INFO - Produit TOY-009-RED a terminé l'étape peinture en 1.68s.
2025-03-11 17:54:12,512 - INFO - Produit TOY-009-RED ajouté à la file d'attente de controle_qualite.
2025-03-11 17:54:12,636 - INFO - Produit TOY-009-RED a terminé l'étape controle_qualite en 1.23s.
2025-03-11 17:54:12,637 - INFO - Produit TOY-009-RED ajouté à la file d'attente de emballage.
2025-03-11 17:54:12,743 - INFO - Produit TOY-009-RED a terminé l'étape emballage en 1.01s.
2025-03-11 17:54:13,003 - INFO - Produit TOY-010-BLUE a terminé l'étape assemblage en 2.56s.
2025-03-11 17:54:13,005 - INFO - Produit TOY-010-BLUE ajouté à la file d'attente de peinture.
2025-03-11 17:54:13,264 - INFO - Produit TOY-010-BLUE a terminé l'étape peinture en 2.54s.
2025-03-11 17:54:13,265 - INFO - Produit TOY-010-BLUE ajouté à la file d'attente de controle_qualite.
2025-03-11 17:54:13,372 - INFO - Produit TOY-010-BLUE a terminé l'étape controle_qualite en 1.05s.
2025-03-11 17:54:13,372 - INFO - Produit TOY-010-BLUE ajouté à la file d'attente de emballage.
2025-03-11 17:54:13,466 - INFO - Produit TOY-010-BLUE a terminé l'étape emballage en 0.89s.
2025-03-11 17:54:13,737 - INFO - Produit TOY-003-BLUE a terminé l'étape assemblage en 2.69s.
2025-03-11 17:54:13,737 - INFO - Produit TOY-003-BLUE ajouté à la file d'attente de peinture.
2025-03-11 17:54:14,192 - INFO - Produit TOY-003-BLUE a terminé l'étape peinture en 4.49s.
2025-03-11 17:54:14,192 - INFO - Produit TOY-003-BLUE ajouté à la file d'attente de controle_qualite.
2025-03-11 17:54:14,336 - INFO - Produit TOY-003-BLUE a terminé l'étape controle_qualite en 1.39s.
2025-03-11 17:54:14,336 - INFO - Produit TOY-003-BLUE ajouté à la file d'attente de emballage.
2025-03-11 17:54:14,434 - INFO - Produit TOY-003-BLUE a terminé l'étape emballage en 0.96s.
2025-03-11 17:58:11,158 - INFO - Produit TOY-001-RED ajouté à la file d'attente de assemblage.
2025-03-11 17:58:11,158 - INFO - Produit TOY-002-RED ajouté à la file d'attente de assemblage.
2025-03-11 17:58:11,158 - INFO - Produit TOY-003-GREEN ajouté à la file d'attente de assemblage.
2025-03-11 17:58:11,158 - INFO - Produit TOY-004-BLUE ajouté à la file d'attente de assemblage.
2025-03-11 17:58:11,158 - INFO - Produit TOY-005-RED ajouté à la file d'attente de assemblage.
2025-03-11 17:58:11,158 - INFO - Produit TOY-006-GREEN ajouté à la file d'attente de assemblage.
2025-03-11 17:58:11,158 - INFO - Produit TOY-007-BLUE ajouté à la file d'attente de assemblage.
2025-03-11 17:58:11,158 - INFO - Produit TOY-008-RED ajouté à la file d'attente de assemblage.
2025-03-11 17:58:11,158 - INFO - Produit TOY-009-GREEN ajouté à la file d'attente de assemblage.
2025-03-11 17:58:11,158 - INFO - Produit TOY-010-RED ajouté à la file d'attente de assemblage.
2025-03-11 17:58:11,298 - INFO - Produit TOY-001-RED a terminé l'étape assemblage en 1.34s.
2025-03-11 17:58:11,298 - INFO - Produit TOY-001-RED ajouté à la file d'attente de peinture.
2025-03-11 17:58:11,575 - INFO - Produit TOY-001-RED a terminé l'étape peinture en 2.75s.
2025-03-11 17:58:11,575 - INFO - Produit TOY-001-RED ajouté à la file d'attente de controle_qualite.
2025-03-11 17:58:11,639 - INFO - Produit TOY-001-RED a terminé l'étape controle_qualite en 0.63s.
2025-03-11 17:58:11,639 - INFO - Produit TOY-001-RED ajouté à la file d'attente de emballage.
2025-03-11 17:58:11,780 - INFO - Produit TOY-001-RED a terminé l'étape emballage en 1.39s.
2025-03-11 17:58:11,998 - INFO - Produit TOY-002-RED a terminé l'étape assemblage en 2.14s.
2025-03-11 17:58:11,999 - INFO - Produit TOY-002-RED ajouté à la file d'attente de peinture.
2025-03-11 17:58:12,282 - INFO - Produit TOY-002-RED a terminé l'étape peinture en 2.77s.
2025-03-11 17:58:12,282 - INFO - Produit TOY-002-RED ajouté à la file d'attente de controle_qualite.
2025-03-11 17:58:12,414 - INFO - Produit TOY-002-RED a terminé l'étape controle_qualite en 1.31s.
2025-03-11 17:58:12,414 - INFO - Produit TOY-002-RED ajouté à la file d'attente de emballage.
2025-03-11 17:58:12,514 - INFO - Produit TOY-002-RED a terminé l'étape emballage en 0.96s.
2025-03-11 17:58:12,786 - INFO - Produit TOY-003-GREEN a terminé l'étape assemblage en 2.67s.
2025-03-11 17:58:12,787 - INFO - Produit TOY-003-GREEN ajouté à la file d'attente de peinture.
2025-03-11 17:58:13,222 - INFO - Produit TOY-003-GREEN a terminé l'étape peinture en 4.32s.
2025-03-11 17:58:13,223 - INFO - Produit TOY-003-GREEN ajouté à la file d'attente de controle_qualite.
2025-03-11 17:58:13,335 - INFO - Produit TOY-003-GREEN a terminé l'étape controle_qualite en 1.06s.
2025-03-11 17:58:13,336 - INFO - Produit TOY-003-GREEN ajouté à la file d'attente de emballage.
2025-03-11 17:58:13,404 - INFO - Produit TOY-003-GREEN a terminé l'étape emballage en 0.64s.
2025-03-11 17:58:13,519 - INFO - Produit TOY-004-BLUE a terminé l'étape assemblage en 1.08s.
2025-03-11 17:58:13,519 - INFO - Produit TOY-004-BLUE ajouté à la file d'attente de peinture.
2025-03-11 17:58:13,692 - INFO - Produit TOY-004-BLUE a terminé l'étape peinture en 1.69s.
2025-03-11 17:58:13,692 - INFO - Produit TOY-004-BLUE ajouté à la file d'attente de controle_qualite.
2025-03-11 17:58:13,768 - INFO - Produit TOY-004-BLUE a terminé l'étape controle_qualite en 0.71s.
2025-03-11 17:58:13,769 - INFO - Produit TOY-004-BLUE ajouté à la file d'attente de emballage.
2025-03-11 17:58:13,916 - INFO - Produit TOY-004-BLUE a terminé l'étape emballage en 1.41s.
2025-03-11 17:58:14,183 - INFO - Produit TOY-005-RED a terminé l'étape assemblage en 2.58s.
2025-03-11 17:58:14,183 - INFO - Produit TOY-005-RED ajouté à la file d'attente de peinture.
2025-03-11 17:58:14,371 - INFO - Produit TOY-005-RED a terminé l'étape peinture en 1.82s.
2025-03-11 17:58:14,374 - INFO - Produit TOY-005-RED ajouté à la file d'attente de controle_qualite.
2025-03-11 17:58:14,475 - INFO - Produit TOY-005-RED a terminé l'étape controle_qualite en 0.99s.
2025-03-11 17:58:14,475 - INFO - Produit TOY-005-RED ajouté à la file d'attente de emballage.
2025-03-11 17:58:14,593 - INFO - Produit TOY-005-RED a terminé l'étape emballage en 1.09s.
2025-03-11 17:58:14,593 - WARNING - La station assemblage est en panne pour 6.38s.
2025-03-11 17:58:14,593 - WARNING - La station assemblage est en panne, aucun produit traité.
2025-03-11 17:58:14,593 - WARNING - La station assemblage est en panne, aucun produit traité.
2025-03-11 17:58:14,593 - WARNING - La station assemblage est en panne, aucun produit traité.
2025-03-11 17:58:14,593 - WARNING - La station assemblage est en panne, aucun produit traité.
2025-03-11 17:58:14,593 - WARNING - La station assemblage est en panne, aucun produit traité.
2025-03-11 17:58:14,594 - WARNING - La station assemblage est en panne, aucun produit traité.
2025-03-11 17:58:14,594 - WARNING - La station assemblage est en panne, aucun produit traité.
2025-03-11 17:58:14,594 - INFO - La station assemblage est réparée et reprend son fonctionnement.
2025-03-11 17:58:14,779 - INFO - Produit TOY-006-GREEN a terminé l'étape assemblage en 1.80s.
2025-03-11 17:58:14,781 - INFO - Produit TOY-006-GREEN ajouté à la file d'attente de peinture.
2025-03-11 17:58:15,146 - INFO - Produit TOY-006-GREEN a terminé l'étape peinture en 3.61s.
2025-03-11 17:58:15,146 - INFO - Produit TOY-006-GREEN ajouté à la file d'attente de controle_qualite.
2025-03-11 17:58:15,208 - INFO - Produit TOY-006-GREEN a terminé l'étape controle_qualite en 0.57s.
2025-03-11 17:58:15,208 - INFO - Produit TOY-006-GREEN ajouté à la file d'attente de emballage.
2025-03-11 17:58:15,289 - INFO - Produit TOY-006-GREEN a terminé l'étape emballage en 0.80s.
2025-03-11 17:58:15,411 - INFO - Produit TOY-007-BLUE a terminé l'étape assemblage en 1.17s.
2025-03-11 17:58:15,411 - INFO - Produit TOY-007-BLUE ajouté à la file d'attente de peinture.
2025-03-11 17:58:15,705 - INFO - Produit TOY-007-BLUE a terminé l'étape peinture en 2.88s.
2025-03-11 17:58:15,706 - INFO - Produit TOY-007-BLUE ajouté à la file d'attente de controle_qualite.
2025-03-11 17:58:15,781 - INFO - Produit TOY-007-BLUE a terminé l'étape controle_qualite en 0.73s.
2025-03-11 17:58:15,781 - INFO - Produit TOY-007-BLUE ajouté à la file d'attente de emballage.
2025-03-11 17:58:15,920 - INFO - Produit TOY-007-BLUE a terminé l'étape emballage en 1.33s.
2025-03-11 17:58:16,081 - INFO - Produit TOY-008-RED a terminé l'étape assemblage en 1.56s.
2025-03-11 17:58:16,082 - INFO - Produit TOY-008-RED ajouté à la file d'attente de peinture.
2025-03-11 17:58:16,305 - INFO - Produit TOY-008-RED a terminé l'étape peinture en 2.18s.
2025-03-11 17:58:16,305 - INFO - Produit TOY-008-RED ajouté à la file d'attente de controle_qualite.
2025-03-11 17:58:16,396 - INFO - Produit TOY-008-RED a terminé l'étape controle_qualite en 0.83s.
2025-03-11 17:58:16,396 - INFO - Produit TOY-008-RED ajouté à la file d'attente de emballage.
2025-03-11 17:58:16,471 - INFO - Produit TOY-008-RED a terminé l'étape emballage en 0.74s.
2025-03-11 17:58:16,653 - INFO - Produit TOY-009-GREEN a terminé l'étape assemblage en 1.76s.
2025-03-11 17:58:16,654 - INFO - Produit TOY-009-GREEN ajouté à la file d'attente de peinture.
2025-03-11 17:58:16,877 - INFO - Produit TOY-009-GREEN a terminé l'étape peinture en 2.16s.
2025-03-11 17:58:16,877 - INFO - Produit TOY-009-GREEN ajouté à la file d'attente de controle_qualite.
2025-03-11 17:58:16,979 - INFO - Produit TOY-009-GREEN a terminé l'étape controle_qualite en 0.92s.
2025-03-11 17:58:16,980 - WARNING - Produit TOY-009-GREEN marqué comme défectueux, ajouté à la pile.
2025-03-11 17:58:16,980 - INFO - Produit TOY-009-GREEN renvoyé à l'étape assemblage pour correction (tentative 2).
2025-03-11 17:58:16,980 - INFO - Produit TOY-009-GREEN ajouté à la file d'attente de assemblage.
2025-03-11 17:58:17,120 - INFO - Produit TOY-010-RED a terminé l'étape assemblage en 1.36s.
2025-03-11 17:58:17,120 - INFO - Produit TOY-010-RED ajouté à la file d'attente de peinture.
2025-03-11 17:58:17,384 - INFO - Produit TOY-010-RED a terminé l'étape peinture en 2.59s.
2025-03-11 17:58:17,384 - INFO - Produit TOY-010-RED ajouté à la file d'attente de controle_qualite.
2025-03-11 17:58:17,440 - INFO - Produit TOY-010-RED a terminé l'étape controle_qualite en 0.53s.
2025-03-11 17:58:17,440 - INFO - Produit TOY-010-RED ajouté à la file d'attente de emballage.
2025-03-11 17:58:17,514 - INFO - Produit TOY-010-RED a terminé l'étape emballage en 0.73s.
2025-03-11 17:58:17,757 - INFO - Produit TOY-009-GREEN a terminé l'étape assemblage en 2.37s.
2025-03-11 17:58:17,757 - INFO - Produit TOY-009-GREEN ajouté à la file d'attente de peinture.
2025-03-11 17:58:17,980 - INFO - Produit TOY-009-GREEN a terminé l'étape peinture en 2.21s.
2025-03-11 17:58:17,980 - INFO - Produit TOY-009-GREEN ajouté à la file d'attente de controle_qualite.
2025-03-11 17:58:18,046 - INFO - Produit TOY-009-GREEN a terminé l'étape controle_qualite en 0.61s.
2025-03-11 17:58:18,047 - WARNING - Produit TOY-009-GREEN marqué comme défectueux, ajouté à la pile.
2025-03-11 17:58:18,048 - INFO - Produit TOY-009-GREEN renvoyé à l'étape peinture pour correction (tentative 3).
2025-03-11 17:58:18,048 - INFO - Produit TOY-009-GREEN ajouté à la file d'attente de peinture.
2025-03-11 17:58:18,338 - INFO - Produit TOY-009-GREEN a terminé l'étape peinture en 2.87s.
2025-03-11 17:58:18,338 - INFO - Produit TOY-009-GREEN ajouté à la file d'attente de controle_qualite.
2025-03-11 17:58:18,466 - INFO - Produit TOY-009-GREEN a terminé l'étape controle_qualite en 1.23s.
2025-03-11 17:58:18,466 - INFO - Produit TOY-009-GREEN ajouté à la file d'attente de emballage.
2025-03-11 17:58:18,546 - INFO - Produit TOY-009-GREEN a terminé l'étape emballage en 0.70s.
2025-03-11 18:20:12,225 - INFO - Produit TOY-001-RED ajouté à la file d'attente de assemblage.
2025-03-11 18:20:12,225 - INFO - Produit TOY-002-BLUE ajouté à la file d'attente de assemblage.
2025-03-11 18:20:12,225 - INFO - Produit TOY-003-RED ajouté à la file d'attente de assemblage.
2025-03-11 18:20:12,225 - INFO - Produit TOY-004-GREEN ajouté à la file d'attente de assemblage.
2025-03-11 18:20:12,226 - INFO - Produit TOY-005-RED ajouté à la file d'attente de assemblage.
2025-03-11 18:20:12,226 - INFO - Produit TOY-006-GREEN ajouté à la file d'attente de assemblage.
2025-03-11 18:20:12,226 - INFO - Produit TOY-007-GREEN ajouté à la file d'attente de assemblage.
2025-03-11 18:20:12,226 - INFO - Produit TOY-008-BLUE ajouté à la file d'attente de assemblage.
2025-03-11 18:20:12,226 - INFO - Produit TOY-009-BLUE ajouté à la file d'attente de assemblage.
2025-03-11 18:20:12,226 - INFO - Produit TOY-010-GREEN ajouté à la file d'attente de assemblage.
2025-03-11 18:20:12,521 - INFO - Produit TOY-001-RED a terminé l'étape assemblage en 2.89s.
2025-03-11 18:20:12,521 - INFO - Produit TOY-001-RED ajouté à la file d'attente de peinture.
2025-03-11 18:20:12,521 - WARNING - La station peinture est en panne pour 8.86s.
2025-03-11 18:20:12,521 - WARNING - La station peinture est en panne, aucun produit traité.
2025-03-11 18:20:12,685 - INFO - Produit TOY-002-BLUE a terminé l'étape assemblage en 1.59s.
2025-03-11 18:20:12,685 - INFO - Produit TOY-002-BLUE ajouté à la file d'attente de peinture.
2025-03-11 18:20:12,685 - WARNING - La station peinture est en panne, aucun produit traité.
2025-03-11 18:20:12,894 - INFO - Produit TOY-003-RED a terminé l'étape assemblage en 2.05s.
2025-03-11 18:20:12,895 - INFO - Produit TOY-003-RED ajouté à la file d'attente de peinture.
2025-03-11 18:20:12,895 - WARNING - La station peinture est en panne, aucun produit traité.
2025-03-11 18:20:13,106 - INFO - Produit TOY-004-GREEN a terminé l'étape assemblage en 2.06s.
2025-03-11 18:20:13,106 - INFO - Produit TOY-004-GREEN ajouté à la file d'attente de peinture.
2025-03-11 18:20:13,106 - WARNING - La station peinture est en panne, aucun produit traité.
2025-03-11 18:20:13,351 - INFO - Produit TOY-005-RED a terminé l'étape assemblage en 2.42s.
2025-03-11 18:20:13,352 - INFO - Produit TOY-005-RED ajouté à la file d'attente de peinture.
2025-03-11 18:20:13,352 - WARNING - La station peinture est en panne, aucun produit traité.
2025-03-11 18:20:13,352 - WARNING - La station assemblage est en panne pour 7.83s.
2025-03-11 18:20:13,352 - WARNING - La station assemblage est en panne, aucun produit traité.
2025-03-11 18:20:13,352 - WARNING - La station peinture est en panne, aucun produit traité.
2025-03-11 18:20:13,352 - WARNING - La station assemblage est en panne, aucun produit traité.
2025-03-11 18:20:13,352 - WARNING - La station peinture est en panne, aucun produit traité.
2025-03-11 18:20:13,352 - WARNING - La station assemblage est en panne, aucun produit traité.
2025-03-11 18:20:13,352 - WARNING - La station peinture est en panne, aucun produit traité.
2025-03-11 18:20:13,352 - WARNING - La station assemblage est en panne, aucun produit traité.
2025-03-11 18:20:13,352 - WARNING - La station peinture est en panne, aucun produit traité.
2025-03-11 18:20:13,352 - WARNING - La station assemblage est en panne, aucun produit traité.
2025-03-11 18:20:13,352 - INFO - La station peinture est réparée et reprend son fonctionnement.
2025-03-11 18:20:13,504 - INFO - Produit TOY-001-RED a terminé l'étape peinture en 1.52s.
2025-03-11 18:20:13,505 - INFO - Produit TOY-001-RED ajouté à la file d'attente de controle_qualite.
2025-03-11 18:20:13,569 - INFO - Produit TOY-001-RED a terminé l'étape controle_qualite en 0.62s.
2025-03-11 18:20:13,569 - INFO - Produit TOY-001-RED ajouté à la file d'attente de emballage.
2025-03-11 18:20:13,718 - INFO - Produit TOY-001-RED a terminé l'étape emballage en 1.45s.
2025-03-11 18:20:13,720 - WARNING - La station assemblage est en panne, aucun produit traité.
2025-03-11 18:20:14,076 - INFO - Produit TOY-002-BLUE a terminé l'étape peinture en 3.51s.
2025-03-11 18:20:14,076 - INFO - Produit TOY-002-BLUE ajouté à la file d'attente de controle_qualite.
2025-03-11 18:20:14,230 - INFO - Produit TOY-002-BLUE a terminé l'étape controle_qualite en 1.43s.
2025-03-11 18:20:14,230 - WARNING - Produit TOY-002-BLUE marqué comme défectueux, ajouté à la pile.
2025-03-11 18:20:14,230 - INFO - Produit TOY-002-BLUE renvoyé à l'étape assemblage pour correction (tentative 2).
2025-03-11 18:20:14,230 - INFO - Produit TOY-002-BLUE ajouté à la file d'attente de assemblage.
2025-03-11 18:20:14,230 - WARNING - La station assemblage est en panne, aucun produit traité.
2025-03-11 18:20:14,527 - INFO - Produit TOY-003-RED a terminé l'étape peinture en 2.91s.
2025-03-11 18:20:14,529 - INFO - Produit TOY-003-RED ajouté à la file d'attente de controle_qualite.
2025-03-11 18:20:14,612 - INFO - Produit TOY-003-RED a terminé l'étape controle_qualite en 0.75s.
2025-03-11 18:20:14,613 - INFO - Produit TOY-003-RED ajouté à la file d'attente de emballage.
2025-03-11 18:20:14,714 - INFO - Produit TOY-003-RED a terminé l'étape emballage en 0.97s.
2025-03-11 18:20:14,715 - WARNING - La station assemblage est en panne, aucun produit traité.
2025-03-11 18:20:15,002 - INFO - Produit TOY-004-GREEN a terminé l'étape peinture en 2.82s.
2025-03-11 18:20:15,002 - INFO - Produit TOY-004-GREEN ajouté à la file d'attente de controle_qualite.
2025-03-11 18:20:15,127 - INFO - Produit TOY-004-GREEN a terminé l'étape controle_qualite en 1.17s.
2025-03-11 18:20:15,129 - WARNING - Produit TOY-004-GREEN marqué comme défectueux, ajouté à la pile.
2025-03-11 18:20:15,129 - INFO - Produit TOY-004-GREEN renvoyé à l'étape peinture pour correction (tentative 2).
2025-03-11 18:20:15,129 - INFO - Produit TOY-004-GREEN ajouté à la file d'attente de peinture.
2025-03-11 18:20:15,129 - INFO - La station assemblage est réparée et reprend son fonctionnement.
2025-03-11 18:20:15,338 - INFO - Produit TOY-006-GREEN a terminé l'étape assemblage en 2.08s.
2025-03-11 18:20:15,339 - INFO - Produit TOY-006-GREEN ajouté à la file d'attente de peinture.
2025-03-11 18:20:15,789 - INFO - Produit TOY-005-RED a terminé l'étape peinture en 4.44s.
2025-03-11 18:20:15,791 - INFO - Produit TOY-005-RED ajouté à la file d'attente de controle_qualite.
2025-03-11 18:20:15,924 - INFO - Produit TOY-005-RED a terminé l'étape controle_qualite en 1.27s.
2025-03-11 18:20:15,924 - INFO - Produit TOY-005-RED ajouté à la file d'attente de emballage.
2025-03-11 18:20:16,027 - INFO - Produit TOY-005-RED a terminé l'étape emballage en 1.02s.
2025-03-11 18:20:16,144 - INFO - Produit TOY-007-GREEN a terminé l'étape assemblage en 1.11s.
2025-03-11 18:20:16,144 - INFO - Produit TOY-007-GREEN ajouté à la file d'attente de peinture.
2025-03-11 18:20:16,409 - INFO - Produit TOY-004-GREEN a terminé l'étape peinture en 2.51s.
2025-03-11 18:20:16,410 - INFO - Produit TOY-004-GREEN ajouté à la file d'attente de controle_qualite.
2025-03-11 18:20:16,557 - INFO - Produit TOY-004-GREEN a terminé l'étape controle_qualite en 1.42s.
2025-03-11 18:20:16,557 - INFO - Produit TOY-004-GREEN ajouté à la file d'attente de emballage.
2025-03-11 18:20:16,677 - INFO - Produit TOY-004-GREEN a terminé l'étape emballage en 1.14s.
2025-03-11 18:20:16,677 - WARNING - La station assemblage est en panne pour 9.00s.
2025-03-11 18:20:16,677 - WARNING - La station assemblage est en panne, aucun produit traité.
2025-03-11 18:20:16,892 - INFO - Produit TOY-006-GREEN a terminé l'étape peinture en 2.02s.
2025-03-11 18:20:16,893 - INFO - Produit TOY-006-GREEN ajouté à la file d'attente de controle_qualite.
2025-03-11 18:20:17,026 - INFO - Produit TOY-006-GREEN a terminé l'étape controle_qualite en 1.27s.
2025-03-11 18:20:17,026 - INFO - Produit TOY-006-GREEN ajouté à la file d'attente de emballage.
2025-03-11 18:20:17,137 - INFO - Produit TOY-006-GREEN a terminé l'étape emballage en 1.06s.
2025-03-11 18:20:17,137 - WARNING - La station assemblage est en panne, aucun produit traité.
2025-03-11 18:20:17,427 - INFO - Produit TOY-007-GREEN a terminé l'étape peinture en 2.89s.
2025-03-11 18:20:17,428 - INFO - Produit TOY-007-GREEN ajouté à la file d'attente de controle_qualite.
2025-03-11 18:20:17,428 - WARNING - La station controle_qualite est en panne pour 8.90s.
2025-03-11 18:20:17,428 - WARNING - La station controle_qualite est en panne, aucun produit traité.
2025-03-11 18:20:17,428 - WARNING - La station assemblage est en panne, aucun produit traité.
2025-03-11 18:20:17,428 - WARNING - La station controle_qualite est en panne, aucun produit traité.
2025-03-11 18:20:17,428 - WARNING - La station assemblage est en panne, aucun produit traité.
2025-03-11 18:20:17,428 - WARNING - La station controle_qualite est en panne, aucun produit traité.
2025-03-11 18:20:17,428 - WARNING - La station assemblage est en panne, aucun produit traité.
2025-03-11 18:20:17,428 - WARNING - La station controle_qualite est en panne, aucun produit traité.
2025-03-11 18:20:17,428 - WARNING - La station assemblage est en panne, aucun produit traité.
2025-03-11 18:20:17,428 - WARNING - La station controle_qualite est en panne, aucun produit traité.
2025-03-11 18:20:17,428 - WARNING - La station assemblage est en panne, aucun produit traité.
2025-03-11 18:20:17,428 - WARNING - La station controle_qualite est en panne, aucun produit traité.
2025-03-11 18:20:17,428 - WARNING - La station assemblage est en panne, aucun produit traité.
2025-03-11 18:20:17,428 - WARNING - La station controle_qualite est en panne, aucun produit traité.
2025-03-11 18:20:17,428 - WARNING - La station assemblage est en panne, aucun produit traité.
2025-03-11 18:20:17,428 - WARNING - La station controle_qualite est en panne, aucun produit traité.
2025-03-11 18:20:17,428 - INFO - La station assemblage est réparée et reprend son fonctionnement.
2025-03-11 18:20:17,623 - INFO - Produit TOY-008-BLUE a terminé l'étape assemblage en 1.89s.
2025-03-11 18:20:17,624 - INFO - Produit TOY-008-BLUE ajouté à la file d'attente de peinture.
2025-03-11 18:20:17,944 - INFO - Produit TOY-008-BLUE a terminé l'étape peinture en 3.14s.
2025-03-11 18:20:17,952 - INFO - Produit TOY-008-BLUE ajouté à la file d'attente de controle_qualite.
2025-03-11 18:20:17,952 - WARNING - La station controle_qualite est en panne, aucun produit traité.
2025-03-11 18:20:18,228 - INFO - Produit TOY-009-BLUE a terminé l'étape assemblage en 2.70s.
2025-03-11 18:20:18,228 - INFO - Produit TOY-009-BLUE ajouté à la file d'attente de peinture.
2025-03-11 18:20:18,502 - INFO - Produit TOY-009-BLUE a terminé l'étape peinture en 2.69s.
2025-03-11 18:20:18,502 - INFO - Produit TOY-009-BLUE ajouté à la file d'attente de controle_qualite.
2025-03-11 18:20:18,502 - INFO - La station controle_qualite est réparée et reprend son fonctionnement.
2025-03-11 18:20:18,619 - INFO - Produit TOY-007-GREEN a terminé l'étape controle_qualite en 1.12s.
2025-03-11 18:20:18,619 - INFO - Produit TOY-007-GREEN ajouté à la file d'attente de emballage.
2025-03-11 18:20:18,731 - INFO - Produit TOY-007-GREEN a terminé l'étape emballage en 1.06s.
2025-03-11 18:20:18,929 - INFO - Produit TOY-010-GREEN a terminé l'étape assemblage en 1.92s.
2025-03-11 18:20:18,930 - INFO - Produit TOY-010-GREEN ajouté à la file d'attente de peinture.
2025-03-11 18:20:19,370 - INFO - Produit TOY-010-GREEN a terminé l'étape peinture en 4.38s.
2025-03-11 18:20:19,371 - INFO - Produit TOY-010-GREEN ajouté à la file d'attente de controle_qualite.
2025-03-11 18:20:19,507 - INFO - Produit TOY-008-BLUE a terminé l'étape controle_qualite en 1.31s.
2025-03-11 18:20:19,507 - INFO - Produit TOY-008-BLUE ajouté à la file d'attente de emballage.
2025-03-11 18:20:19,584 - INFO - Produit TOY-008-BLUE a terminé l'étape emballage en 0.72s.
2025-03-11 18:20:19,851 - INFO - Produit TOY-002-BLUE a terminé l'étape assemblage en 2.64s.
2025-03-11 18:20:19,851 - INFO - Produit TOY-002-BLUE ajouté à la file d'attente de peinture.
2025-03-11 18:20:20,222 - INFO - Produit TOY-002-BLUE a terminé l'étape peinture en 3.65s.
2025-03-11 18:20:20,223 - INFO - Produit TOY-002-BLUE ajouté à la file d'attente de controle_qualite.
2025-03-11 18:20:20,297 - INFO - Produit TOY-009-BLUE a terminé l'étape controle_qualite en 0.70s.
2025-03-11 18:20:20,298 - INFO - Produit TOY-009-BLUE ajouté à la file d'attente de emballage.
2025-03-11 18:20:20,387 - INFO - Produit TOY-009-BLUE a terminé l'étape emballage en 0.84s.
2025-03-11 18:20:20,519 - INFO - Produit TOY-010-GREEN a terminé l'étape controle_qualite en 1.28s.
2025-03-11 18:20:20,520 - INFO - Produit TOY-010-GREEN ajouté à la file d'attente de emballage.
2025-03-11 18:20:20,592 - INFO - Produit TOY-010-GREEN a terminé l'étape emballage en 0.67s.
2025-03-11 18:20:20,673 - INFO - Produit TOY-002-BLUE a terminé l'étape controle_qualite en 0.75s.
2025-03-11 18:20:20,673 - WARNING - Produit TOY-002-BLUE marqué comme défectueux, ajouté à la pile.
2025-03-11 18:20:20,673 - INFO - Produit TOY-002-BLUE renvoyé à l'étape peinture pour correction (tentative 3).
2025-03-11 18:20:20,673 - INFO - Produit TOY-002-BLUE ajouté à la file d'attente de peinture.
2025-03-11 18:20:21,069 - INFO - Produit TOY-002-BLUE a terminé l'étape peinture en 3.91s.
2025-03-11 18:20:21,070 - INFO - Produit TOY-002-BLUE ajouté à la file d'attente de controle_qualite.
2025-03-11 18:20:21,178 - INFO - Produit TOY-002-BLUE a terminé l'étape controle_qualite en 1.03s.
2025-03-11 18:20:21,178 - INFO - Produit TOY-002-BLUE ajouté à la file d'attente de emballage.
2025-03-11 18:20:21,293 - INFO - Produit TOY-002-BLUE a terminé l'étape emballage en 1.09s.
```